# Introduction

Follow this tutorial to get to know the most important features of allms!



# Import and utils

In [1]:
# This allows to run asynchronous code in a Jupyter notebook
import nest_asyncio

nest_asyncio.apply()

## Setting up your LLM

To start working with `allms` you need to import one of the supported models and configure it. Make sure to have access to an Azure OpenAI endpoint and dispose of the needed information. In this tutorial we are going to use a GPT model.

In [3]:
from allms.models import AzureOpenAIModel
from allms.domain.configuration import AzureOpenAIConfiguration

configuration = AzureOpenAIConfiguration(
    api_key="your-secret-api-key",
    base_url="https://endpoint.openai.azure.com/",
    api_version="2023-03-15-preview",
    deployment="gpt-35-turbo",
    model_name="gpt-3.5-turbo"
)

model = AzureOpenAIModel(config=configuration)

## Basic usage

The model has a `generate()` method that is responsible for running the generations. In the most basic case, you can simply provide it with a prompt and it’ll return generated content. 

In [5]:
model.generate("What is the capital of Poland?")

[ResponseData(response='The capital of Poland is Warsaw.', input_data=None, number_of_prompt_tokens=7, number_of_generated_tokens=7, error=None)]

This was an example of the most basic usage. But what if you wanted to run a single prompt multiple times, but with slightly changed data? For example, you have a dataset of reviews and you want to classify each of them as positive or negative. You can use batch mode to do this.

## Batch mode

Let's say you have a dataset with 3 reviews and you want to classify each of them as positive or negative. To do so:
- create a `prompt` and inside it use symbolic variable `{review}`, which will later be replaced by actual reviews coming from the dataset.
- create `input_data`. `input_data` is simply a list of `InputData`, where each `InputData` is a single example and it's a dataclass with two fields:
  - `input_mappings` - a dictionary mapping symbolic variables used in the prompt to the actual review.
  - `id` - is needed because requests are made asynchronously, so the output order will not always be the same as the input order.
- run the generation by calling the `generate()` method with the `prompt` and `input_data` as arguments. 

This will automatically run the generation in async mode, so it'll be much faster than a normal, sequential calling. Additionally, it'll automatically retry requests in case of failure. 

In [6]:
from allms.domain.input_data import InputData


positive_review_0 = "Very good coffee, lightly roasted, with good aroma and taste. The taste of sourness is barely noticeable (which is good because I don't like sour coffees). After grinding, the aroma spreads throughout the room. I recommend it to all those who do not like strongly roasted and pitch-black coffees. A very good solution is to close the package with string, which allows you to preserve the aroma and freshness."
positive_review_1 = "Delicious coffee!! Delicate, just the way I like it, and the smell after opening is amazing. It smells freshly roasted. Faithful to Lavazza coffee for years, I decided to look for other flavors. Based on the reviews, I blindly bought it and it was a 10-shot, it outperformed Lavazze in taste. For me the best."
negative_review = "Marketing is doing its job and I was tempted too, but this coffee is nothing above the level of coffees from the supermarket. And the method of brewing or grinding does not help here. The coffee is simply weak - both in terms of strength and taste. I do not recommend."

prompt = "You'll be provided with a review of a coffe. Decide if the review is positive or negative. Review: {review}"
input_data = [
    InputData(input_mappings={"review": positive_review_0}, id="0"),
    InputData(input_mappings={"review": positive_review_1}, id="1"),
    InputData(input_mappings={"review": negative_review}, id="2")
]

responses = model.generate(prompt=prompt, input_data=input_data)

{f"review_id={response.input_data.id}": response.response for response in responses}

{'review_id=0': 'The review is positive.',
 'review_id=1': 'The review is positive.',
 'review_id=2': 'The review is negative.'}

### Multiple symbolic variables

The example above showed a prompt with only one symbolic variable used in it. But you can use as many of them as you want.

Let’s say you have two reviews: one positive and one negative, and you want the model to tell which one of them is positive. To do so:
- create a prompt as shown in the cell below. Two symbolic variables are used inside it: `{first_review}` and `{second_review}`.
- create `input_data`. It looks similar to the example above - it's a list of `InputData`, but here the `input_mappings` fields have two entries, one per single symbolic variable used in the prompt.
- same as above, generation is ran by calling the `generate()` method.

In [7]:
prompt = """You'll be provided with two reviews of a coffee. Decide which one is positive.

First review: {first_review}
Second review: {second_review}"""
input_data = [
    InputData(input_mappings={"first_review": positive_review_0, "second_review": negative_review}, id="0"),
    InputData(input_mappings={"first_review": negative_review, "second_review": positive_review_1}, id="1"),
]

responses = model.generate(prompt=prompt, input_data=input_data)
{f"example_id={response.input_data.id}": response.response for response in responses}

{'example_id=0': 'The first review is positive.',
 'example_id=1': 'The second review is positive.'}

## Forcing model response format

This is one of the most interesting features of our library. In a production setup, it's often the case that we want the model to return generated content in a format that will later be easy to ingest by the rest of our pipeline - for example, json with some predefined fields. With our library it’s really easy to achieve.

Let’s say that again you have a review of a coffee, and you want the model to generate information that might be interesting for you, and additionally you want it to return them in the format provided by you. To do so, first you have to create a dataclass that defines the output format and the information you want the model to generate. Each field of this dataclass must have a type defined and also a description provided that describes what given field means. The better the description, the better the model will understand what it should generate for a given field.

In [8]:
import typing
    
from pydantic import BaseModel, Field
    
class ReviewOutputDataModel(BaseModel):
    summary: str = Field(description="Summary of a product description")
    should_buy: bool = Field(description="Recommendation whether I should buy the product or not")
    brand_name: str = Field(description="Brand of the coffee")
    aroma:str = Field(description="Description of the coffee aroma")
    cons: typing.List[str] = Field(description="List of cons of the coffee")

The next thing is to create a prompt, which can be pretty simple as shown in the cell below, and the `input_data` for the model. To force the model to generate a response in a given format, you have to call the `generate()` method with `prompt`, `input_data` and with one additional argument called `output_data_model_class`. The `ReviewOutputDataModel` class defined above should be provided to this argument. This automatically tells the model to output predictions in the format defined by this dataclass.

In [9]:
review = "Marketing is doing its job and I was tempted too, but this Blue Orca coffee is nothing above the level of coffees from the supermarket. And the method of brewing or grinding does not help here. The coffee is simply weak - both in terms of strength and taste. I do not recommend."
    
prompt = "Summarize review of the coffee. Review: {review}"
input_data = [
    InputData(input_mappings={"review": review}, id="0")
]

responses = model.generate(
    prompt=prompt, 
    input_data=input_data,
    output_data_model_class=ReviewOutputDataModel
)
response = responses[0].response

The results below show that the predictions are indeed returned in the format defined above. 

In [10]:
type(response)

__main__.ReviewOutputDataModel

In [11]:
response.dict()

{'summary': 'The Blue Orca coffee is nothing above the level of coffees from the supermarket. It is weak in terms of strength and taste.',
 'should_buy': False,
 'brand_name': 'Blue Orca',
 'aroma': 'Not mentioned in the review',
 'cons': ['Weak in terms of strength', 'Weak in terms of taste']}

In [12]:
response

ReviewOutputDataModel(summary='The Blue Orca coffee is nothing above the level of coffees from the supermarket. It is weak in terms of strength and taste.', should_buy=False, brand_name='Blue Orca', aroma='Not mentioned in the review', cons=['Weak in terms of strength', 'Weak in terms of taste'])

This is really interesting feature, because it gives the possibility to do several tasks at once. In the above example, there was summarization, classification, entity extraction and so on. To add another one, simply add a new field to the dataclass. For example, if you'd like to know the pros of the coffee, you just need to add one additional field `pros` to the dataclass, describe it properly, re-run everything and you'll get the results. So as you can see, it significantly reduces the need to do extensive prompt engineering. You just define it in the code as an additional field and you’re done.